# 端到端案例研究: 用于渠道网络的分层MPC-PID控制系统

该脚本构建并运行一个包含两个闸门和三个渠道的串联仿真系统，完全按照详细的设计规范。它展示了一个分层分布式控制架构，其中一个中央MPC智能体为两个本地PID控制器设定目标。

系统的核心目标是在应对上游入流扰动时，将每个闸门前的水位维持在动态调整的最优设定点。

### 工作流程:
1. **预测**: 一个“全知”的预测智能体发布对未来入流的完美预测。
2. **正常运行 (0-100秒)**: MPC将设定点维持在正常水平。PID控制器跟踪这些设定点。
3. **扰动 (100-200秒)**: 一个扰动智能体注入额外入流。
4. **MPC响应**: 中央MPC智能体接收到预测，并计算出更低的“应急”设定点以预先释放库容。
5. **PID响应**: 本地PID控制器接收到新的设定点，并调整闸门开度以达到这些新目标。
6. **恢复 (200秒后)**: 扰动结束，MPC逐渐将设定点恢复到正常水平。

### 1. 导入库并加载配置

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 导入所有平台组件
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.physical_objects.canal import Canal
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.io.physical_io_agent import PhysicalIOAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.simulation_identification.disturbances.rainfall_agent import RainfallAgent
from swp.local_agents.prediction.inflow_forecaster_agent import InflowForecasterAgent
from swp.central_coordination.dispatch.central_mpc_agent import CentralMPCAgent
from swp.core.interfaces import Agent

CONFIG_PATH = 'examples/hierarchical_mpc_control_case_study.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义辅助组件与函数

In [ ]:
class DataLoggerAgent(Agent):
    """一个简单的智能体，用于记录来自特定主题的数据以供绘图。"""
    def __init__(self, agent_id, bus, topics_to_log):
        super().__init__(agent_id)
        self.bus = bus
        self.topics_to_log = topics_to_log
        self.latest_values = {name: None for name in topics_to_log.values()}
        self.log = []
        for topic, name in self.topics_to_log.items():
            self.bus.subscribe(topic, lambda msg, t=name: self._update_latest(msg, t))

    def _update_latest(self, message, topic_name):
        self.latest_values[topic_name] = message.get('new_setpoint')

    def run(self, current_time: float):
        for topic_name, value in self.latest_values.items():
            if value is not None:
                self.log.append({'time': current_time, 'topic': topic_name, 'value': value})

def setup_system(bus: MessageBus, cfg: dict):
    """设置仿真所需的所有物理组件和智能体。"""
    sim_dt = cfg['simulation']['dt']
    comp_cfgs = cfg['components']
    
    UPSTREAM_STATE_TOPIC = "state/canal/upstream"
    DOWNSTREAM_STATE_TOPIC = "state/canal/downstream"
    UPSTREAM_CMD_TOPIC = "command/pid1/setpoint"
    DOWNSTREAM_CMD_TOPIC = "command/pid2/setpoint"
    GATE1_ACTION_TOPIC = "action/gate1/opening"
    GATE2_ACTION_TOPIC = "action/gate2/opening"
    INFLOW_FORECAST_TOPIC = "forecast/inflow"
    DISTURBANCE_TOPIC = cfg['disturbance']['topic']

    # 物理组件
    canal_params = comp_cfgs['canal']
    normal_water_level = cfg['mpc']['normal_setpoint_upstream']
    surface_area = canal_params['length'] * (canal_params['bottom_width'] + canal_params['side_slope_z'] * normal_water_level)
    initial_volume = surface_area * normal_water_level
    gate_params = comp_cfgs['gate']

    upstream_pool = Canal("upstream_canal", {'volume': initial_volume, 'water_level': normal_water_level}, canal_params, bus, DISTURBANCE_TOPIC)
    control_gate_1 = Gate("control_gate_1", {'opening': 1.0}, gate_params)
    downstream_pool = Canal("downstream_canal", {'volume': initial_volume, 'water_level': normal_water_level}, canal_params)
    final_gate_2 = Gate("final_gate_2", {'opening': 1.0}, gate_params)
    sink_canal = Canal("sink_canal", {'volume': 0, 'water_level': 0}, comp_cfgs['sink_canal'])
    components = [upstream_pool, control_gate_1, downstream_pool, final_gate_2, sink_canal]

    # 智能体
    io_agent = PhysicalIOAgent("io_agent_main", bus, {}, {'gate1': {'obj': control_gate_1, 'target_attr': 'target_opening', 'topic': GATE1_ACTION_TOPIC, 'control_key': 'control_signal'}, 'gate2': {'obj': final_gate_2, 'target_attr': 'target_opening', 'topic': GATE2_ACTION_TOPIC, 'control_key': 'control_signal'}})
    twin_upstream = DigitalTwinAgent("twin_upstream", upstream_pool, bus, UPSTREAM_STATE_TOPIC)
    twin_downstream = DigitalTwinAgent("twin_downstream", downstream_pool, bus, DOWNSTREAM_STATE_TOPIC)

    pid1 = PIDController(setpoint=cfg['mpc']['normal_setpoint_upstream'], min_output=0, max_output=gate_params['max_opening'], **cfg['pid'])
    pid2 = PIDController(setpoint=cfg['mpc']['normal_setpoint_downstream'], min_output=0, max_output=gate_params['max_opening'], **cfg['pid'])
    lca1 = LocalControlAgent("lca1", pid1, bus, UPSTREAM_STATE_TOPIC, 'water_level', GATE1_ACTION_TOPIC, sim_dt, UPSTREAM_CMD_TOPIC)
    lca2 = LocalControlAgent("lca2", pid2, bus, DOWNSTREAM_STATE_TOPIC, 'water_level', GATE2_ACTION_TOPIC, sim_dt, DOWNSTREAM_CMD_TOPIC)

    forecaster_config = {**cfg['forecaster'], "forecast_topic": INFLOW_FORECAST_TOPIC, "dt": sim_dt, "prediction_horizon": cfg['mpc']['prediction_horizon']}
    forecaster = InflowForecasterAgent("forecaster", bus, forecaster_config)
    rainfall_agent = RainfallAgent("rainfall", bus, cfg['disturbance'])

    mpc_config = {**cfg['mpc'], "dt": sim_dt, "state_keys": ['upstream_level', 'downstream_level'], "state_subscriptions": {'upstream_level': UPSTREAM_STATE_TOPIC, 'downstream_level': DOWNSTREAM_STATE_TOPIC}, "forecast_subscription": INFLOW_FORECAST_TOPIC, "command_topics": {'upstream_cmd': UPSTREAM_CMD_TOPIC, 'downstream_cmd': DOWNSTREAM_CMD_TOPIC}, "normal_setpoints": [cfg['mpc']['normal_setpoint_upstream'], cfg['mpc']['normal_setpoint_downstream']], "canal_surface_areas": [surface_area] * 2}
    central_dispatcher = CentralMPCAgent("dispatcher", bus, mpc_config)

    logger = DataLoggerAgent("logger", bus, {UPSTREAM_CMD_TOPIC: "upstream_setpoint", DOWNSTREAM_CMD_TOPIC: "downstream_setpoint"})
    agents = [io_agent, twin_upstream, twin_downstream, lca1, lca2, rainfall_agent, forecaster, central_dispatcher, logger]

    return components, agents, logger

def plot_results(harness, logger, cfg):
    history = harness.history
    results = pd.DataFrame()
    if history:
        flat_history = []
        for step in history:
            row = {'time': step['time']}
            for name, states in step.items():
                if name != 'time' and isinstance(states, dict):
                    for key, value in states.items(): row[f"{name}.{key}"] = value
            flat_history.append(row)
        results = pd.DataFrame(flat_history).set_index('time')

    if logger.log:
        log_df = pd.DataFrame(logger.log).set_index('time')
        setpoint_df = log_df.pivot(columns='topic', values='value')
        results = results.join(setpoint_df, how='outer').ffill()

    fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True)
    fig.suptitle('分层MPC控制案例研究结果', fontsize=16)

    axes[0].plot(results.index, results['upstream_canal.water_level'], label='上游渠道水位')
    axes[0].plot(results.index, results['downstream_canal.water_level'], label='下游渠道水位')
    if 'upstream_setpoint' in results.columns: axes[0].plot(results.index, results['upstream_setpoint'], '--', label='上游设定点 (来自MPC)')
    if 'downstream_setpoint' in results.columns: axes[0].plot(results.index, results['downstream_setpoint'], '--', label='下游设定点 (来自MPC)')
    axes[0].axhline(y=cfg['mpc']['normal_setpoint_upstream'], color='grey', linestyle=':', label='正常设定点')
    axes[0].axhline(y=cfg['mpc']['emergency_setpoint'], color='red', linestyle=':', label='紧急设定点')
    dist_start = cfg['disturbance']['start_time']
    axes[0].axvline(x=dist_start, color='k', linestyle='--', label='扰动开始')
    axes[0].axvline(x=dist_start + cfg['disturbance']['duration'], color='k', linestyle='--')
    axes[0].set_title('渠道水位与MPC设定点'); axes[0].set_ylabel('水位 (m)'); axes[0].legend(); axes[0].grid(True)

    axes[1].plot(results.index, results['control_gate_1.opening'], label='水闸1开度')
    axes[1].plot(results.index, results['final_gate_2.opening'], label='水闸2开度')
    axes[1].set_title('水闸开度'); axes[1].set_ylabel('开度 (m)'); axes[1].legend(); axes[1].grid(True)

    axes[2].plot(results.index, results['upstream_canal.outflow'], label='上游渠道出流')
    axes[2].plot(results.index, results['downstream_canal.outflow'], label='下游渠道出流')
    dist_inflow = [cfg['disturbance']['inflow_rate'] if dist_start <= t < dist_start + cfg['disturbance']['duration'] else 0 for t in results.index]
    axes[2].plot(results.index, dist_inflow, label='扰动入流', color='red', linestyle='-.')
    axes[2].set_title('流量'); axes[2].set_ylabel('流量 (m³/s)'); axes[2].set_xlabel('时间 (s)'); axes[2].legend(); axes[2].grid(True)

    plt.tight_layout()
    plt.show()

### 3. 运行案例研究

In [ ]:
message_bus = MessageBus()
components, agents, logger = setup_system(message_bus, config)

harness = SimulationHarness(config['simulation'])
for component in components: harness.add_component(component)
for agent in agents: harness.add_agent(agent)

harness.add_connection("upstream_canal", "control_gate_1")
harness.add_connection("control_gate_1", "downstream_canal")
harness.add_connection("downstream_canal", "final_gate_2")
harness.add_connection("final_gate_2", "sink_canal")
harness.build()

print("\n--- 开始运行仿真 ---")
harness.run_mas_simulation()
print("\n--- 仿真结束 ---")

plot_results(harness, logger, config)